In [ ]:
import panel as pn

from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
from bokeh.plotting import figure, output_file, save, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColorBar,ColumnDataSource,OpenURL,TapTool
from bokeh.palettes import YlOrRd9
from bokeh.transform import linear_cmap
from bokeh.models.tools import HoverTool
from bokeh.events import Tap

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

from pyproj import Proj, transform

import pandas as pd
import numpy as np
import datetime as dt

pn.extension()

In [ ]:
dT_levels=['+0.5C','+1.0C','+1.5C','+2.0C','+2.5C','+3.0C','+3.5C']

CRBCPI_data={dT_levels[0]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+0.5C_NBCC.xls"),
             dT_levels[1]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+1.0C_NBCC.xls"),
             dT_levels[2]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+1.5C_NBCC.xls"),
             dT_levels[3]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+2.0C_NBCC.xls"),
             dT_levels[4]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+2.5C_NBCC.xls"),
             dT_levels[5]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+3.0C_NBCC.xls"),
             dT_levels[6]:pd.read_excel("https://climate-scenarios.canada.ca/files/buildings_report/Appendix_1.2_NBCC/Appendix1.2_+3.5C_NBCC.xls")}

CRBCPI_dT_to_time=pd.DataFrame([[2023,2023,2023,2023],
                                [2035,2046,2046,np.nan],
                                [2047,2070,2070,np.nan],
                                [2059,2087,np.nan,np.nan],
                                [2069,np.nan,np.nan,np.nan],
                                [2080,np.nan,np.nan,np.nan],
                                [2090,np.nan,np.nan,np.nan]],
                                index=dT_levels,columns=['RCP8.5','RCP6.0','RCP4.5','RCP2.6'])

In [ ]:
plot_width=1000
plot_height=1000

In [ ]:
t1='''Before we begin, it is important that you are comfortable with certain definitions and concepts related to climate models and future climate data.'''
t2='''Do you want to learn more about:'''

information_sources={'Future climate change scenarios':'https://climatedata.ca/resource/introduction-to-decision-making-using-climate-scenarios/)', 
                     'Understanding the range of potential climate change projections':'https://climatedata.ca/resource/understanding-ranges-in-climate-projections/', 
                     'Navigating the ClimateData.ca climate data portal':'https://climatedata.ca/resource/how-to-navigate-variable-maps/', 
                     'The Government of Canada Climate-Resilience Buildings and Core Public Infrastructure program?':'https://www.infrastructure.gc.ca/plan/crbcpi-irccipb-eng.html',
                     'The Government of Canada Climate Lens program?':'https://www.infrastructure.gc.ca/pub/other-autre/cl-occ-eng.html',
                     'The Federation of Canadian Municipalities for Climate Innovation program?':'https://fcm.ca/en/programs/municipalities-climate-innovation-program/climate-change-adaptation'
                     }

knowledge_checkbox = pn.widgets.CheckBoxGroup(inline=False,
                                                name='Core knowledge assessment',
                                                value=[],
                                                options=information_sources
                                             )

selected = pn.pane.Markdown(object='')

def knowledge_checkbox_callback(*events):
    print(events)
    for event in events:
        if event.name == 'options':
            selections.object = 'Possible options: %s' % ', '.join(event.new)
        elif event.name == 'value':
            selected.object = 'Selected: %s' % ','.join(event.new)

##Register the callback
watcher = knowledge_checkbox.param.watch(knowledge_checkbox_callback, ['options', 'value'], onlychanged=False)

##Initialize the callback


Tab_Core_Knowledge_Checklist=pn.Column(t1, 
                                       t2, 
                                       knowledge_checkbox,
                                       selected,
                                       width=plot_width, height=plot_height, 
                                       name='Core Knowledge Checklist')


In [ ]:
building_type = pn.widgets.TextInput(name='What type of building are you designing or operating?', placeholder='Enter building type here...')

building_lifespan = pn.widgets.DateRangeSlider(
    name='What is your building lifespan?',
    start=dt.datetime(1950,1,1), end=dt.datetime(2100, 1,1),
    value=(dt.datetime(2021, 1,1), dt.datetime(2061, 1,1))
    )

proj1 = Proj('epsg:4326', preserve_units=False)
proj2 = Proj('epsg:3785', preserve_units=False)

bbox_lat=[41.,71.]
bbox_lon=[360.-142.,360.-51.]
bbox_x,bbox_y=transform(proj1,proj2,bbox_lat,bbox_lon)

coordList=[]

tile_provider = get_provider(CARTODBPOSITRON)
TOOLS="hover,pan,wheel_zoom,tap"

location_map = figure(plot_width=500, plot_height=250,x_range=bbox_x, y_range=bbox_y,title='Where is your building located?',
          x_axis_type="mercator", y_axis_type="mercator",tools=TOOLS)
location_map.add_tile(tile_provider)

source = ColumnDataSource(data=dict(x=[0], y=[0]))   
location_map.circle(source=source,x='x',y='y')

def map_location_callback(event):
    Coords=(event.x,event.y)
    coordList.append(Coords) 
    source.data = dict(x=[i[0] for i in coordList], y=[i[1] for i in coordList])

location_map.on_event(Tap, map_location_callback)

profession_type = pn.widgets.TextInput(name='What is your profession or role?', placeholder='Enter profession/role here...')

Tab_Project_Definition=pn.Column(building_type,
                                 building_lifespan,
                                 profession_type,
                                 pn.pane.Bokeh(location_map),
                                 width=plot_width, height=plot_height,
                                 name='Project Definition')

In [ ]:
building_components=["foundations",
             "basements",
             "superstructure (floor and roof)",
             "exterior closures (exterior wall, windows, doors)",
             "roofing (coverings and openings",
             "Doors and partitions",
             "staircases",
             "interior finishes (wall, floor and ceiling finishes)",
             "conveying systems (e.g. elevators)",
             "Plumbing (water supply, sewage, drainage, etc.)",
             "Mechnical heating systems",
             "Mechanical cooling systems",
             "Additional/specialized HVAC systems",
             "Fire protection (includes sprinklers and hoses)",
             "Electrical (includes service, distribution, power and comms)"
             ]

building_components_widget = pn.widgets.multi_choice = pn.widgets.MultiChoice(name='Which building components would you like to include in this assessment?', value=[],
             options=building_components)

#TODO: add ability to add new items

Tab_Building_Component_Inventory=pn.Column(building_components_widget,
                                           width=plot_width, height=plot_height,
                                           name='Building Component Inventory')

In [ ]:
climate_hazards=["marine coastal flooding",
                 "river and lake spring flooding",
                 "extreme rain flooding",
                 "ice jam flooding",
                 "extreme snow",
                 "extreme cold",
                 "permafrost loss",
                 "wildfire"
                 ]
climate_hazards_widget = pn.widgets.multi_choice = pn.widgets.MultiChoice(name='Which climate hazards is your building potentially vulnerable to, if the hazard occurred now or in the future?', value=[],
             options=climate_hazards)

#TODO: add ability to add new options

Tab_Climate_Hazard_Inventory=pn.Column(climate_hazards_widget,
                                       width=plot_width, height=plot_height,
                                       name='Climate Hazard Inventory')

In [100]:
#TODO: make axes of matrix respond to the selections made for building components and climate hazards
#TODO: make accordion of slider boxes, that allow individual pixel values of map (vulnerability levels) to be updated
## https://holoviews.org/user_guide/Responding_to_Events.html
## https://holoviews.org/user_guide/Custom_Interactivity.html

data=pd.DataFrame(columns=climate_hazards, index=building_components) #set up dataframe
data.loc[:,:]=0 #initialize all values to zero
heatmap_data=pd.DataFrame(data.stack(), columns=['vulnerability']).reset_index()#
heatmap_data.rename(columns={"level_0":"Building Component","level_1":"Climate Hazard"})

def vulnerability_heatmap(vulnerability_level=0):
    heatmap_data['vulnerability'][2,4]=vulnerability_level
    #heatmap_data_list=list(heatmap_data.to_records(index=False))
    hm = hv.HeatMap(heatmap_data)
    hm.opts(opts.HeatMap(width=900,height=900,tools=[],toolbar='above',
                         xrotation=45,colorbar=True,
                         clim=(0,10), cmap='magma'))
    return hm
dmap = hv.DynamicMap(vulnerability_heatmap, kdims=['vulnerability_level'])

Tab_Hazard_Component_Interactions=pn.Column(dmap.redim.range(vulnerability_level=(0,10)),
                                            width=plot_width, height=plot_height,
                                            name='Vulnerability Matrix')

In [101]:
tabs=pn.Tabs(Tab_Core_Knowledge_Checklist,    
             Tab_Project_Definition,
             Tab_Building_Component_Inventory,
             Tab_Climate_Hazard_Inventory,
             Tab_Hazard_Component_Interactions)
tabs

Tabs
    [0] Column(height=1000, name='Core Knowledge C..., sizing_mode='fixed', width=1000)
        [0] Markdown(str)
        [1] Markdown(str)
        [2] CheckBoxGroup(name='Core knowledge a..., options={'Future climate change sc...})
        [3] Markdown(str)
    [1] Column(height=1000, name='Project Definition', sizing_mode='fixed', width=1000)
        [0] TextInput(name='What type of b..., placeholder='Enter building t...)
        [1] DateRangeSlider(end=datetime.datetime(2100, ..., name='What is your b..., start=datetime.datetime(1950, ..., value=(datetime.datetime(2021, ..., value_end=datetime.datetime(2061, ..., value_start=datetime.datetime(2021, ...)
        [2] TextInput(name='What is your p..., placeholder='Enter profession/role h...)
        [3] Bokeh(Figure)
    [2] Column(height=1000, name='Building Component I..., sizing_mode='fixed', width=1000)
        [0] MultiChoice(name='Which building c..., options=['foundations', ...])
    [3] Column(height=1000, name='Climate Hazard I..., sizing_mode='fixed', width=1000)
        [0] MultiChoice(name='Which climate hazards is ..., options=['marine coastal flooding'...])
    [4] Column(height=1000, name='Vulnerability Matrix', sizing_mode='fixed', width=1000)
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] IntSlider(end=10, margin=(20, 20, 20, 20), name='vulnerability_level', width=250)
                [1] VSpacer()

In [ ]:
#https://holoviews.org/reference/elements/bokeh/Sankey.html#elements-bokeh-gallery-sankey